<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## 4.05 - Generalized Linear Models

_Authors: Matt Brems (DC) h/t Timothy Book, Justin Pounders (ATL)_

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline

## Generalized Linear Models (GLMs)

**Generalized linear models** describes a class of models that take the linear model (think linear regression) and generalizes the linear model beyond the assumptions of the linear regression model we discussed last week.

In fact, you have already seen an example of a _generalized_ linear model: logistic regression.

**Linear regression:**

$$\hat{y} = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p $$

> The predicted variable $\hat{y}$ will be in the range $(-\infty, \infty)$.

**Logistic regression:**

$$ \hat{y} = \sigma\left(\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p\right) $$

where

$$ \sigma(t) = \frac{e^t}{1 + e^t} $$

> The predicted variable $\hat{y}$ is now in the range $(0, 1)$.

---

> **What are examples of things we might predict that don't fall in the range $(-\infty, \infty)$ or $(0,1)$?**

### Bend the spoon...

Generalized linear models are _linear in the coefficients_ but "bend" the linear estimator to match the range we want for our target, $Y$.

Three ingredients:
- The linear piece
- The "bending" function
- A probability distribution for errors

Putting these three components together leads to something that looks generically like...

$$ Y = g\left( \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p \right) + \varepsilon $$
where

- $\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p$ is the linear piece
- $g(\cdot)$ is the bending piece; called a link function
- $\varepsilon$ is the random "error" piece

---

_Let's look at two specific examples that you have already seen!_

---

**Linear regression:**

- Linear piece: $\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p$
- Link function: $g(t) = t$
- Error component: $\varepsilon \sim N(0,\sigma)$

![](./images/lin_reg.png)

**Logistic regression:**

- Linear piece: $\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p$
- Link function: $g(t) = \frac{e^t}{1 + e^t}$
- Error component: $\varepsilon \sim Bernoulli(p)$

![](./images/log_reg.png)

### Choices, choices, choices...

Chosing the right kind of generalized linear model (GLM) from all possibilities really boils down to picking the "error."

The "error" model is really telling you how you expect observations to be distributed.  It is a probability distribution.

> 1. In traditional linear regression, the error term is a normal distribution.  This means that you expect actual observations to be normally distributed around your line.

> 2. In logistic regression, the error term is a Bernoulli distribution.  This means that you expect actual observations to be above (1) or below (0) the logit curve with a certain probability.

Choosing the distribution function often points to a link function you should use: [here is a table](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function).

Examples:

1. If $Y$ is a non-negative integer:
   - Poisson regression if mean $\approx$ variance
   - Negative Binomial regression if variance $\gg$ mean (overdisperse)
   - For example,
     - Units sold
     - Customers through the door
     - Patients to the ER
     - Number of cars racing the red light
2. If $Y$ values represent categories
   - Multinomial logistic regression (unordered categories)
   - Ordinal logstic regression (ordered categories)
   - For example,
     - Does a population tend to buy groceries at Whole Foods, Publix or Kroger?
     - Will millenials vote democrat, republican or independent?
     - Predicting the Amazon star rating of books.
3. If $Y$ values are continuous, non-negative
   - Gamma regression
   - For example,
     - How long before my Uber/Lyft gets here?
     - Home prices

## The `statsmodels` API

We will use the `statsmodels` API to explore GLMs in Python.  (`sklearn` does not have a robust implementation for GLMs.)  Documentation and examples for `statsmodels` can be found [here](http://www.statsmodels.org/stable/generated/statsmodels.genmod.generalized_linear_model.GLM.html#statsmodels.genmod.generalized_linear_model.GLM).

## Logistic Regression

**When do we use it?** When we want to model something on the $\{0,1\}$ range... like the probability that a person will develop a disease.

> Is logistic regression actually _regression_ or _classification_???

### The Data
The data used from this example come from one of UCLA's IDRE modules (for R).  The module can be found [here](https://stats.idre.ucla.edu/r/dae/logit-regression/).

**Data Description:** _A researcher is interested in how variables, such as GRE (Graduate Record Exam scores), GPA (grade point average) and prestige of the undergraduate institution, effect admission into graduate school. The response variable, admit/don’t admit, is a binary variable._

In [ ]:
grad = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

# Let's do some EDA!
print(grad.shape)
grad.head()

In [ ]:
grad.describe()

In [ ]:
grad.info()

In [ ]:
grad.admit.value_counts()

In [ ]:
grad.boxplot('gre', by='admit')
grad.boxplot(['gpa', 'rank'], by='admit')

**Model time...**

In [ ]:
# Now, let's build our GLM


**Check:** How would I interpret the coefficient for `rank`?

**Interpretation:** As the rank of my undergraduate institution increases by 1, my odds of being admitted drop by a factor of 0.57.

**Check:** If I scored 100 points better on my GRE, how would that affect my odds of acceptance?

**Interpretation:** As my GRE score increases by 100 points, my odds of being admitted go up by 26%.

**Note:**
- GLMs are typically not "directly solvable."  Instead, a solution is initially guessed then iteratively refined until to you converge on an answer.
- The default maximum number of iterations for GLMs is 100. If `No. Iterations` is 100, that means the algorithm probably didn't converge and that the $\mathbf{\hat{\beta}}$ are still changing. Therefore, **your output is unreliable - DO NOT USE IT**.

## Poisson Regression

**When do we use it?** When we want to model something on the $\{0,1,2,\ldots\}$ range... like number of objects sold or nunber of awards earned!

#### Data
We'll again rely on UCLA's IDRE module.  This one can be found [here](https://stats.idre.ucla.edu/r/dae/poisson-regression/).

#### Data Description
_The number of awards earned by students at one high school. Predictors of the number of awards earned include the type of program in which the student was enrolled (e.g., vocational, general or academic) and the score on their final exam in math._

In [ ]:
award = pd.read_csv("https://stats.idre.ucla.edu/stat/data/poisson_sim.csv")

# Let's do some EDA:
award.head()

In [ ]:
award.plot('math', 'num_awards', kind = 'scatter')
award.boxplot('num_awards', by = 'prog')

In [ ]:
# Notice that prog is actually a categorical variable - I am aware of this.
# I'm going to suspsend that knowledge for the sake of example.


**Interpretation**: For a one-unit increase in `math`, I expect to win $e^{0.0861} \approx 1.09$ times as many awards.

## Gamma Regression

**When do we use it?** When we want to model something on the $[0,\infty)$ range... like time until some event occurs!

### The Data
The data used from this example come from a 1945 study about and is inspired by [Peter Craigmile's use](http://www.craigmile.com/peter/teaching/7430/notes/7_gamma_influence.pdf) of this example.

**Data Description:** _“Hurn, et al. (1945) published data on the clotting time of blood, giving clotting time in seconds for normal plasma diluted to nine different percentage concentrations with prothrombin-free plasma; clotting was induced by two lots of thromboplastin.” [McCullagh and Nelder](http://www.utstat.toronto.edu/~brunner/oldclass/2201s11/readings/glmbook.pdf)_

In [ ]:
clot = pd.read_csv("./clotting.csv", index_col = "index")

In [ ]:
clot.head()

In [ ]:
clot.boxplot('clot_time', by = 'lot')

In [ ]:
clot.plot('plasma_pct', 'clot_time', kind = 'scatter')

In [ ]:
# Feature engineering, anyone?

**Interpretation**: For a one-unit increase in `plasma_pct_log10`, I expect the blood will take $e^{-1.4654} \approx 23\%$ as much time to clot.

In [ ]:
math.exp(-1.4654)

---

## Analysis of Deviance (BONUS)

We've spoken briefly about deviance before as a generalization of the sums of squares of error for generalized linear models.

Suppose we have two models:

$$
\begin{eqnarray*}
Y_{full} &=& \beta_0 + \beta_1X_1 + \cdots + \beta_kX_k + \cdots + \beta_pX_p + \varepsilon \\
Y_{reduced} &=& \beta_0 + \beta_1X_1 + \cdots + \beta_kX_k + \varepsilon
\end{eqnarray*}
$$

We say that $Y_{reduced}$ is nested in $Y_{full}$, because the reduced model could "fit inside" the full. (You can learn more about nested linear regression models [here](http://people.reed.edu/~jones/Courses/P24.pdf), although the ideas approximately hold for generalized linear models as well.)

When we have one model nested inside the other, there is a statistical test to see if adding new variables are worth it. (Think about it like looking at the difference in mean squared error or $R^2$ by adding a variable, but getting a $p$-value quantifying whether or not it's worth it!)

We calculate the **deviance** of the reduced model and subtract the **deviance** of the full model from it. This difference in deviance follows a [chi-squared distribution](https://en.wikipedia.org/wiki/Chi-squared_distribution) with $p-k$ degrees of freedom. (Note that $p-k$ indicates how many variables we took out of our full model to get to the reduced model!)

**This comparison only works with nested models! Do not use if your models are not nested!**

In [ ]:
# Test model differences
from scipy.stats import chi2

# First, build our top model
indep_vars = ['gre', 'gpa', 'rank']
X = sm.add_constant(grad[indep_vars])
y = grad.admit

glm_logit = sm.GLM(y, 
                   X,
                   sm.families.Binomial(sm.families.links.logit))
results_logit = glm_logit.fit()


# Next, let's see if we can safely reduce our model
reduced_vars = ['gre', 'gpa']
X_reduced = sm.add_constant(grad[reduced_vars])

results_reduced = sm.GLM(y,
                 X_reduced,
                 sm.families.Binomial(sm.families.links.logit)).fit()
results_reduced.summary()


# Calculate the difference in deviance
D = results_reduced.deviance - results_logit.deviance
print('Difference in Deviance: ', D)

# Check to see if this difference is significant
pval = 1 - chi2.cdf(D, df = 1)
print('p-value of test of difference: ', pval) # What can we conclude here?

$H_0:$ reduced model is better

$H_A:$ reduced model is not better

Because $p < \alpha$, we reject $H_0$ and conclude that the reduced model is not better.